In [1]:
# !pip install findspark
# !pip install pyspark

In [2]:
# import findspark
# findspark.init()

In [3]:
# #Initializing PySpark
# from pyspark import SparkContext, SparkConf

# # #Spark Config
# conf = SparkConf().setAppName("sample_app")
# sc = SparkContext(conf=conf)

In [4]:
%time
from pyspark.sql import SparkSession
#sc.stop()
spark = SparkSession.builder\
        .master("local[2]")\
        .appName("8INF919D1")\
        .getOrCreate()
       

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/13 08:43:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/13 08:43:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/13 08:43:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
%time
from pyspark import SparkContext
sc = spark.sparkContext.getOrCreate()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [6]:
%time
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, 'iris_libsvm.txt')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


In [7]:
%time
data.first()

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 26 µs


LabeledPoint(0.0, (4,[0,1,2,3],[5.1,3.5,1.4,0.2]))

In [9]:
dfl = data.toDF()
dfl.printSchema()
dfl.show()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)



+--------------------+-----+
|            features|label|
+--------------------+-----+
|(4,[0,1,2,3],[5.1...|  0.0|
|(4,[0,1,2,3],[4.9...|  0.0|
|(4,[0,1,2,3],[4.7...|  0.0|
|(4,[0,1,2,3],[4.6...|  0.0|
|(4,[0,1,2,3],[5.0...|  0.0|
|(4,[0,1,2,3],[5.4...|  0.0|
|(4,[0,1,2,3],[4.6...|  0.0|
|(4,[0,1,2,3],[5.0...|  0.0|
|(4,[0,1,2,3],[4.4...|  0.0|
|(4,[0,1,2,3],[4.9...|  0.0|
|(4,[0,1,2,3],[5.4...|  0.0|
|(4,[0,1,2,3],[4.8...|  0.0|
|(4,[0,1,2,3],[4.8...|  0.0|
|(4,[0,1,2,3],[4.3...|  0.0|
|(4,[0,1,2,3],[5.8...|  0.0|
|(4,[0,1,2,3],[5.7...|  0.0|
|(4,[0,1,2,3],[5.4...|  0.0|
|(4,[0,1,2,3],[5.1...|  0.0|
|(4,[0,1,2,3],[5.7...|  0.0|
|(4,[0,1,2,3],[5.1...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [10]:
%time
(trainingData, testData) = data.randomSplit([0.7, 0.3])

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.72 µs


In [11]:
%time
model = DecisionTree.trainClassifier(
    trainingData, 
    numClasses=3, 
    categoricalFeaturesInfo={},
    impurity='entropy', 
    maxDepth=5, 
    maxBins=32)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [ ]:
%time
print(model.toDebugString())

In [ ]:
%time
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification tree model:')
